In [1]:
import pypelines.supervised_pipeline as pipe
from pypelines import utils

In [2]:
utils.list_supported_models(model_type='classification')

['Decision Tree',
 'Logistic Regression',
 'Random Forest',
 'SVC',
 'XGBoost',
 'MLP',
 'Ridge Classifier',
 'Perceptron Classifier',
 'SGD Classifier',
 'GBT Classifier',
 'ADABoost Classifier',
 'ExtraTrees Classifier',
 'PassiveAggressive Classifier',
 'LDA Classifier',
 'QDA Classifier',
 'NuSVC Classifier',
 'GaussianNB Classifier',
 'MultinomialNB Classifier',
 'ComplementNB Classifier',
 'BernoulliNB Classifier',
 'CategoricalNB Classifier']

In [3]:
import pandas as pd
titanic = pd.read_csv("pypelines/datasets/classification/titanic.csv")

### regression - all models

In [4]:
# code output
reg_pypelines_all = pipe.SupervisedPipeline(data = titanic,target = 'Survived'
                            , model_type = 'classification'
                            ,models = ['Random Forest']
                            , nfolds = 5)

In [5]:
reg_pypelines_all.get_hyperparameters()

{'Random Forest': {'numerical': [{'search': True,
    'name': 'n_estimators',
    'min': 10,
    'max': 100,
    'step': 20},
   {'search': True, 'name': 'max_depth', 'min': 2, 'max': 10, 'step': 2},
   {'search': True,
    'name': 'min_samples_split',
    'min': 0.5,
    'max': 1,
    'step': 0.1},
   {'search': True,
    'name': 'min_samples_leaf',
    'min': 1,
    'max': 10,
    'step': 2}],
  'categorical': [{'search': False,
    'name': 'criterion',
    'selected': ['gini'],
    'values': ['gini', 'entropy']},
   {'search': False,
    'name': 'max_features',
    'selected': ['auto'],
    'values': ['auto', 'sqrt', 'log2']},
   {'search': False,
    'name': 'bootstrap',
    'selected': [True],
    'values': [True, False]},
   {'search': False,
    'name': 'oob_score',
    'selected': [True],
    'values': [True, False]},
   {'search': False,
    'name': 'warm_start',
    'selected': [False],
    'values': [True, False]},
   {'search': False,
    'name': 'class_weight',
    'select

In [6]:
reg_pypelines_all.model_list()

['Random Forest']


In [7]:
reg_pypelines_all.get_code()

Random Forest

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import plotly.express as px
import plotly.graph_objects as go


# target dataframe: titanic
target = "Survived"
features = list(titanic.columns.drop("Survived"))
feature_df = titanic[features]

# get numerical and categorical columns
bool_cols = feature_df.select_dtypes(include=['bool']).columns.tolist()
titanic[bool_cols] = feature_df[bool_cols].astype(int)
numerical_cols = feature_df.select_dtypes(include=['int', 'float']).columns.tolist()
categorical_cols = feature_df.select_dtypes(include=['object']).columns.tolist()
text_cols = feature_df.

In [8]:
rf_dict = {'Random Forest': {'numerical': [{'search': True,
    'name': 'n_estimators',
    'min': 10,
    'max': 100,
    'step': 20},
   {'search': True, 'name': 'max_depth', 'min': 2, 'max': 200, 'step': 2},
   {'search': True,
    'name': 'min_samples_split',
    'min': 0.5,
    'max': 1,
    'step': 0.25},
   {'search': True,
    'name': 'min_samples_leaf',
    'min': 1,
    'max': 10,
    'step': 2}],
  'categorical': [{'search': True,
    'name': 'criterion',
    'selected': ['gini'],
    'values': ['gini', 'entropy']},
   {'search': True,
    'name': 'max_features',
    'selected': ['auto'],
    'values': ['auto', 'sqrt', 'log2']},
   {'search': True,
    'name': 'bootstrap',
    'selected': [True],
    'values': [True, False]},
   {'search': False,
    'name': 'oob_score',
    'selected': [True],
    'values': [True, False]},
   {'search': True,
    'name': 'warm_start',
    'selected': [False],
    'values': [True, False]},
   {'search': True,
    'name': 'class_weight',
    'selected': ['balanced'],
    'values': ['balanced', 'balanced_subsample']}]}}

In [13]:
import json
d = json.loads(rf_dict)

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [12]:
hyperparams = []

for k, v in rf_dict.items():
    for p in v:
        if p.get('search')==False:
            continue
        if k=='categorical':
            if not p['selected']:
                continue
            print(k,v)

AttributeError: 'str' object has no attribute 'get'

In [11]:
type(rf_dict)

dict

In [10]:
reg_pypelines_all.set_grid_search_for_model(model_name = ['Random Forest'],hyperparam_dict = rf_dict )

Random Forest


AttributeError: 'str' object has no attribute 'get'

In [13]:
reg_pypelines_all.code_to_clipboard()

Decision Tree
Logistic Regression
Random Forest
XGBoost
Ridge Classifier
GBT Classifier


In [7]:
reg_pypelines_all.code_to_file(path='./code_output/')

'model files saved to ./code_output/'

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
['Sex', 'Cabin', 'Embarked']
['Name', 'Ticket']
Fitting 5 folds for each of 320 candidates, totalling 1600 fits


KeyboardInterrupt: 

### classification - all models